<a href="https://colab.research.google.com/github/kbotnen/pythonkurs_h25/blob/main/git_kode/Pythonkurs%20-%20Del%203%20-%20Python%20og%20Excel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python workshop - Introduksjon til python - Del 3
Filbehandling og regneark.

## Litt kode for å åpne, endre og lukke filer

In [ ]:
# Open a normal text file.
with open("Pythonkurs - Del 3 - Test read.txt", "r", encoding="utf-8") as file:
    content = file.read()

# Examine the content.
print(content)


In [ ]:
# Write to a normal text file. Use mode 'a' if you want to append instead of overwriting.
with open("Pythonkurs - Del 3 - Test write.txt", "w", encoding="utf-8") as file:
    file.write("First line to write.\n")
    file.write("The second line to write.")


In [ ]:
# Open a Excel file.
import pandas as pd
# Remember: 'conda install openpyxl' before importing xlsx

# Read all sheets into a dictionary, sheet_name=None results in all sheets being read.
all_sheets = pd.read_excel("Pythonkurs - Del 3 - Prices clean.xlsx", sheet_name=None)

In [ ]:
print(type(all_sheets)) # Verify what datastructure we have.
print(all_sheets.keys()) # Let's get the name of the sheets.

In [ ]:
for item in all_sheets.values(): # Let us inspect the results.
    print(type(item))
    print(item)
    print("---")

Vi ser at vi nå har lest inn en excelfil som har to ark i seg. Hvert av de to arkene er represent som en Pandas DataFrame, og er omkapslet av en Dictionary. La oss droppe den overordnede strukturen og få direkte tilgang til hvert av de to arkene.

In [ ]:
df_prices_clean = all_sheets.get("prices_clean", "Not found")
df_makes = all_sheets.get("makes", "Not found")

In [ ]:
df_prices_clean

In [ ]:
df_makes

In [ ]:
df_makes.rename(columns={"Unnamed: 1":"model"}, inplace=True)

In [ ]:
df_makes

In [ ]:
# We can customize settings in Panda. To avoid scientific notation.
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [ ]:
df_prices_clean.groupby("make").price.min()

In [ ]:
df_prices_clean.groupby("make").price.max()

In [ ]:
df_prices_clean.groupby("make").price.mean()

In [ ]:
df_prices_clean.groupby("make").describe()

En av funksjonene som brukes ofte i Excel er pivottabeller. Selv om vi på en måte har gjort det samme i eksemplene over ved hjelp av groupby() så har Pandas en egen pivot_table() metode vi kan se på og.

In [ ]:
# Aggregate on "Symbol". The default aggregation function is 'mean'.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
pivot = pd.pivot_table(data=df_prices_clean, index='make', values=["price"])
pivot

In [ ]:
# Aggreagate the sum instead of the default mean.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
pivot = pd.pivot_table(data=df_prices_clean, index='make', values=["price"], aggfunc='sum')
pivot

In [ ]:
# Use multiple aggregate functions.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
pivot = pd.pivot_table(data=df_prices_clean, index='make', values=["price"], aggfunc=['mean', 'sum'])
pivot

In [ ]:
# Different aggregate functions for each column.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
pivot = pd.pivot_table(data=df_prices_clean, index='make', values=["price", "year"], aggfunc={"price": 'mean', "year": 'count'})
pivot

In [ ]:
# Adding totals to our pivot.
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
pivot = pd.pivot_table(data=df_prices_clean, index='make', values=["price", "year"], aggfunc={"price": 'mean', "year": 'count'}, margins=True)
pivot

In [ ]:
# Sort our result by chaining the sort_values().
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html
pivot = pd.pivot_table(data=df_prices_clean, index='make', values=["price", "year"], aggfunc={"price": 'mean', "year": 'count'}, margins=True).sort_values(by="year")
pivot

In [ ]:
# This will fail in Google Colab due to lack of copy / paste mechanism.
pivot.to_clipboard(excel=True) # We can copy our dataframe to the clipboard!

## Skrive til en Excelfil v.h.a Python
Vi har lest inn en fil, sett litt på innholdet og nå gjenstår det kun å skrive innhold tilbake til filen.

In [ ]:
df_prices_clean.to_excel("Pythonkurs - Del 3 - Python og excel.xlsx", sheet_name='Bilpriser')

In [ ]:
# Write our pivots to Excel.
original_pivot = pivot.copy(deep=True)
pivot.to_excel("Pythonkurs - Del 3 - Python og excel.xlsx", sheet_name='Bilpriser pivot')

In [ ]:
# Write carprices to a csv. Just to show that you can write a DataFrame to csv as well.
df_prices_clean.to_csv("Pythonkurs - Del 3 - Python og excel.csv")

In [ ]:
# Add a new column with new data.
pivot["Mine prisforventninger 2025"] = ((5 * pivot["price"])/100) + pivot["price"]
pivot

In [ ]:
# Write our updated DataFrame to Excel, we want it as a new Sheet.
pivot.to_excel("Pythonkurs - Del 3 - Python og excel.xlsx", sheet_name='Bilpriser pivot forventninger')

In [ ]:
with pd.ExcelWriter("Pythonkurs - Del 3 - Python og excel.xlsx") as writer:
    original_pivot.to_excel(writer, sheet_name='Bilpriser pivot')
    pivot.to_excel(writer, sheet_name='Bilpriser pivot forventninger')

Da har vi sett på hvordan vi kan lese, manipulere og skrive Excelfiler v.h.a Python.

## Oppgave
Fortsett med programmet fra tidligere. Vi skal nå opprette to nye faner i et Excel ark.

* Den ene fanen inneholder selve datasettet vårt. Kastnummer og resultat.
* Den andre fanen inneholder deskriptiv statistikk om datasettet vårt.

NB! Det vil være enklest om du oppretter en Pandas DataFrame basert på datasettet vårt først.